In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
df = pd.read_csv("C:/Users/NOOMAN KHAN/Downloads/stock_nifty50.csv")
df.head()

,Unnamed: 0,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
0,0,2018-12-12,ADANIENT,EQ,151.90,152.8,157.00,151.05,156.00,156.10,154.50,4377945,6.763819e+13,25254,230979,0.0528
1,1,2018-12-13,ADANIENT,EQ,156.10,156.0,158.60,154.00,154.15,156.00,156.58,3499313,5.479299e+13,20136,241770,0.0691
2,2,2018-12-14,ADANIENT,EQ,156.00,156.0,157.90,154.10,157.00,157.10,155.90,3389589,5.284529e+13,19917,235044,0.0693
3,3,2018-12-17,ADANIENT,EQ,157.10,157.1,158.90,156.35,158.00,157.95,157.60,1595605,2.514683e+13,11049,133437,0.0836
4,4,2018-12-18,ADANIENT,EQ,157.95,157.7,159.75,156.20,158.10,158.05,158.14,1689582,2.671824e+13,10488,117265,0.0694


In [3]:
df = df.drop("Unnamed: 0" , axis = 1)

In [4]:
df["Date"] = pd.to_datetime(df["Date"])

In [6]:
pip install tensorflow==1.2.0 --ignore-installed

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow==1.2.0 (from versions: 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0)
ERROR: No matching distribution found for tensorflow==1.2.0


In [12]:
name_of_stock = input("Enter Name Of Stock")
number_of_share = int(input("Number Of Share"))
Y = df[df["Symbol"]==name_of_stock][["Close"]]
start_date = input("Start Date")
end_date = input("End Date")
start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)



from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
Y=scaler.fit_transform(np.array(Y).reshape(-1,1))



training_size=int(len(Y)*0.65)
test_size=len(Y)-training_size
train_data,test_data=Y[0:training_size,:],Y[training_size:len(Y),:1]


import numpy
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return numpy.array(dataX), numpy.array(dataY)


time_step = 50
X_train, y_train = create_dataset(train_data, time_step)
X_test, ytest = create_dataset(test_data, time_step)


X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

### Create the Stacked LSTM model

import tensorflow as tfl
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(50,1)))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')
model.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=50,batch_size=64,verbose=1)


last_date = pd.to_datetime("2022-12-12")
lst_output_1=[]
lst_output_2=[]

diff_1 = start_date-last_date
diff_2 = end_date-last_date
if int(diff_1.days)>0:
    n_steps=50
    x_input=test_data[len(test_data)-n_steps:].reshape(1,-1)
    temp_input=list(x_input)
    temp_input=temp_input[0].tolist()
    i=0
    while(i<int(diff_1.days)):
        if(len(temp_input)>50):
            #print(temp_input)
            x_input=np.array(temp_input[1:])
#                 print("{} day input {}".format(i,x_input))
            x_input=x_input.reshape(1,-1)
            x_input = x_input.reshape((1, n_steps, 1))
            #print(x_input)
            yhat = model.predict(x_input, verbose=0)
#                 print("{} day output {}".format(i,yhat))
            temp_input.extend(yhat[0].tolist())
            temp_input=temp_input[1:]
            #print(temp_input)
            lst_output_1.extend(yhat.tolist())
            i=i+1
        else:
            x_input = x_input.reshape((1, n_steps,1))
            yhat = model.predict(x_input, verbose=0)
#                 print(yhat[0])
            temp_input.extend(yhat[0].tolist())
#                 print(len(temp_input))
            lst_output_1.extend(yhat.tolist())
            i=i+1


        
if int(diff_2.days)>0:
    n_steps=50
    x_input=test_data[len(test_data)-n_steps:].reshape(1,-1)
    temp_input=list(x_input)
    temp_input=temp_input[0].tolist()
    i=0
    while(i<int(diff_2.days)):
        if(len(temp_input)>50):
            #print(temp_input)
            x_input=np.array(temp_input[1:])
#                 print("{} day input {}".format(i,x_input))
            x_input=x_input.reshape(1,-1)
            x_input = x_input.reshape((1, n_steps, 1))
            #print(x_input)
            yhat = model.predict(x_input, verbose=0)
#                 print("{} day output {}".format(i,yhat))
            temp_input.extend(yhat[0].tolist())
            temp_input=temp_input[1:]
            #print(temp_input)
            lst_output_2.extend(yhat.tolist())
            i=i+1
        else:
            x_input = x_input.reshape((1, n_steps,1))
            yhat = model.predict(x_input, verbose=0)
#                 print(yhat[0])
            temp_input.extend(yhat[0].tolist())
#                 print(len(temp_input))
            lst_output_2.extend(yhat.tolist())
            i=i+1
        
invests = scaler.inverse_transform(lst_output_1)[-1]
returns = scaler.inverse_transform(lst_output_2)[-1]
Final_Amount = ((returns-invests)*number_of_share)

if Final_Amount>0:
    print("Profit: " ,Final_Amount)
else:
    print("Loss: " ,Final_Amount)


Enter Name Of StockWIPRO
Number Of Share20
Start Date2022-12-16
End Date2023-04-16
Epoch 1/50
10/10 [==============================] - 11s 359ms/step - loss: 0.0422 - val_loss: 0.0058
Epoch 2/50
10/10 [==============================] - 1s 108ms/step - loss: 0.0082 - val_loss: 0.0107
Epoch 3/50
10/10 [==============================] - 1s 104ms/step - loss: 0.0042 - val_loss: 0.0043
Epoch 4/50
10/10 [==============================] - 1s 106ms/step - loss: 0.0024 - val_loss: 0.0148
Epoch 5/50
10/10 [==============================] - 1s 105ms/step - loss: 0.0021 - val_loss: 0.0073
Epoch 6/50
10/10 [==============================] - 1s 107ms/step - loss: 0.0018 - val_loss: 0.0028
Epoch 7/50
10/10 [==============================] - 1s 106ms/step - loss: 0.0018 - val_loss: 0.0029
Epoch 8/50
10/10 [==============================] - 1s 106ms/step - loss: 0.0017 - val_loss: 0.0091
Epoch 9/50
10/10 [==============================] - 1s 105ms/step - loss: 0.0017 - val_loss: 0.0040
Epoch 10/50
10/1